To do:

- adjust prices etc
- complete list of 

In [1]:
import scrapy
import pandas as pd
import urllib.parse as urlparse
from urllib.parse import parse_qs

class ImmoSpider(scrapy.Spider):
    name = 'immo_spider'
    custom_settings = {
        'ROBOTSTXT_OBEY': True,
        'DOWNLOAD_DELAY': 3, # to avoid '429 Too Many Requests'
        'RETRY_TIMES': 10,
        'AUTOTHROTTLE_ENABLED': True,
        'AUTOTHROTTLE_START_DELAY': 5,
        'AUTOTHROTTLE_MAX_DELAY': 60,
        'AUTOTHROTTLE_TARGET_CONCURRENCY': 1.0
    }

    def __init__(self, flat_house=None, rent_buy=None, place_lvl=None, place=None, *args, **kwargs):
        super(ImmoSpider, self).__init__(*args, **kwargs)
        self.start_urls = [f'https://www.immoscout24.ch/de/{flat_house}/{rent_buy}/{place_lvl}-{place}']
        self.data = []  # List to store data for DataFrame

    def parse(self, response):
        # Navigate through each listing on the page to gather links to detail pages
        links = response.css('a.HgCardElevated_content_uir_2.HgCardElevated_link_EHfr7::attr(href)').getall()
        for link in links:
            url = response.urljoin(link)
            yield scrapy.Request(url, callback=self.parse_details)

        # Follow the pagination link to the next page of listings
        next_page = response.css('a.HgPaginationSelector_nextPreviousArrow__Mlz2[aria-label="Zur nächsten Seite"]::attr(href)').get()
        if next_page:
            yield response.follow(next_page, callback=self.parse)

    def parse_details(self, response):
        # Extract detailed information from each listing's detail page
        price = response.xpath("//div[@data-test='costs']//dd/strong/span/text()").extract_first(default='').replace('CHF', '').replace('’', '')
        street = response.css("address.AddressDetails_address_i3koO .AddressDetails_street_nXScL::text").get()
        plz_city = response.css("address.AddressDetails_address_i3koO span:not(.AddressDetails_street_nXScL)::text").get()
        features_list = response.css('ul.FeaturesFurnishings_list_S54KV li p::text').getall()

                
        # Extracting core attributes from the details page
        core_attributes = response.css("div.CoreAttributes_coreAttributes_e2NAm dl dt")
        details = {
            'type_search': flat_house,
            'place_search': place, 
            'street': street,
            'PLZ': plz_city,
            'price': price,
            'features': ', '.join(features_list),
            'availability': '',
            'object_type': '',
            'number_of_rooms': '',
            'number_of_floors': '',
            'living_area': '',
            'land_area': '',
            'volume': '',
            'construction_year': '',
            'last_rennovation': '',
            'number_of_appartments': '',
            'height_rooms':''
        }
        

        for dt in core_attributes:
            label = dt.xpath("normalize-space(text())").get()
            value = dt.xpath("following-sibling::dd[1]/text()").get() or dt.xpath("following-sibling::dd[1]/span/text()").get()
            if label == 'Verfügbarkeit:':
                details['availability'] = value.strip()
            elif label == 'Objekttyp:':
                details['object_type'] = value.strip()
            elif label == 'Anzahl Zimmer:':
                details['number_of_rooms'] = value.strip()
            elif label == 'Anzahl Etagen:':
                details['number_of_floors'] = value.strip()
            elif label == 'Anzahl Wohnungen:':
                details['number_of_appartments'] = value.strip()
            elif label == 'Wohnfläche:':
                details['living_area'] = value.strip()
            elif label == 'Grundstückfläche:':
                details['land_area'] = value.strip()
            elif label == 'Kubatur:':
                details['volume'] = value.strip()
            elif label == 'Baujahr:':
                details['construction_year'] = value.strip()
            elif label == 'Letztes Renovationsjahr:':
                details['last_rennovation'] = value.strip()
            elif label == 'Raumhöhe:':
                details['height_rooms'] = value.strip()
                
        # Append the extracted data to the list
        self.data.append(details)

        # Follow pagination if necessary
        next_page = response.css('a.HgPaginationSelector_nextPreviousArrow__Mlz2[aria-label="Zur nächsten Seite"]::attr(href)').get()
        if next_page:
            yield response.follow(next_page, self.parse_details)


    def closed(self, reason):
        # When spider is closed, convert collected data to a DataFrame and print/save it
        df = pd.DataFrame(self.data)
        print(df)
        df.to_csv('immobilien_appenzell-ar.csv', index=False)  # Optionally save to CSV
        
from scrapy import crawler
from scrapy.utils.project import get_project_settings

process = crawler.CrawlerProcess(get_project_settings())

# Example parameters
flat_house = "immobilien"
rent_buy = "kaufen"
place_lvl = "kanton"
place = "appenzell-ar"


process.crawl(ImmoSpider, flat_house=flat_house, rent_buy=rent_buy, place=place, place_lvl=place_lvl)
process.start()
process.closed()


2024-05-12 13:16:36 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: scrapybot)
2024-05-12 13:16:36 [scrapy.utils.log] INFO: Versions: lxml 4.9.3.0, libxml2 2.10.4, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.10.0, Python 3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)], pyOpenSSL 23.2.0 (OpenSSL 3.0.13 30 Jan 2024), cryptography 41.0.3, Platform Windows-10-10.0.22631-SP0
2024-05-12 13:16:36 [scrapy.crawler] INFO: Overridden settings:
{'AUTOTHROTTLE_ENABLED': True,
 'DOWNLOAD_DELAY': 3,
 'RETRY_TIMES': 10,
 'ROBOTSTXT_OBEY': True}
2024-05-12 13:16:36 [py.warnings] WARNING: C:\Users\jonas\anaconda3\Lib\site-packages\scrapy\utils\request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is 

    type_search  place_search                 street                PLZ  \
0    immobilien  appenzell-ar      Schmiedgasse 30,        9100 Herisau   
1    immobilien  appenzell-ar    Kurvenstrasse 12a,      9053 Teufen AR   
2    immobilien  appenzell-ar  Teufener Strasse 38,       9042 Speicher   
3    immobilien  appenzell-ar                   None       9064 Hundwil   
4    immobilien  appenzell-ar      Battenhaus 3092,   9052 Niederteufen   
..          ...           ...                    ...                ...   
143  immobilien  appenzell-ar     Hoheggstrasse 13,           9056 Gais   
144  immobilien  appenzell-ar   Kasernenstrasse 30,        9100 Herisau   
145  immobilien  appenzell-ar            Weid 1225,   9428 Walzenhausen   
146  immobilien  appenzell-ar              Herisau,        9100 Herisau   
147  immobilien  appenzell-ar            Teufen AR,      9053 Teufen AR   

           price                                           features  \
0     4380480.–   Balkon / T

AttributeError: 'CrawlerProcess' object has no attribute 'closed'